# PRUEBA DE MODELOS - EMBEDDING Y CLASIFICACIÓN

Lo que se pretende en este libro de Jupyter es empezar con las pruebas de algunos modelos que se pudieron obtener de referencia en investigaciones anteriores. Todo esto con el fin de poder tener una mejor noción de estos modelos y en el mejor de los casos, ya tener un modelo definido con el que se trabajará posteriormente. 

Algunos de estos modelos son simplemente de embedding, otros tantos si conllevan algunas técnicas ya avanzadas de clasificación o detección de imágenes, según corresponda. La lista de modelos que se pretende probar es la siguiente:

- OpenL3 (Solo de embedding)

- ViT (de Google)

- Contrastors: Aquí podemos hacer el uso a su vez de dos modelos, CLIP y MRL

- Sports: Lista de varios modelos que nos pueden ayudar con la tarea asignada

- ResNet: Modelo moldeable con el número de capas que éste utiliza, es una red neuronal convolucional

- InceptionV3: Modelo moldeable con el número de capas y parámetros que éste utiliza, es una red neuronal convolucional

## Modelo 1: OpenL3

Las especificaciones de este modelo se pueden encontrar en la página de [GitHub](https://github.com/marl/openl3?tab=readme-ov-file) o su [Documentación](https://openl3.readthedocs.io/en/latest/tutorial.html#introduction) oficial.


Los parámetros para los métodos de este modelo son los siguientes:

- content_type: "env", "music" (default). "music" es para videos, imágenes o música

- input_repr: "linear", "mel128" (default), "mel256"

- embedding_size: 512, 8192 (default). Tamaño del array resultante con el embedding de la imagen

Si el embedding ya existe, entonces no crea uno nuevo, deja el "original"

Para este modelo, existen 3 posibilidades:

1. Puedes ejecutar el modelo directamente a una imagen (o lista de imágenes) con el método "get_image_embedding"

2. Puedes guardar el embedding en la misma carpeta de donde viene la imagen para un uso posterior. Para guardar el embedding es el método "process_image_file" y para cargarlo es el método "np.load" con np la librería "numpy"

3. Puedes pre cargar desde un principio el modelo para que no estés cargandolo cada que lo requieras para una imagen. El método para pre cargar el modelo es "openl3.models.load_image_embedding_model", después, para usarlo en los métodos de los puntos anteriores, pasas el modelo con el argumento "model"

- Del método "imread" obtuvimos un array de matrices sobre la imagen

- De los métodos de openl3, los argumentos significan lo siguiente: **FALTA INVESTIGAR ESTA PARTE**

- Hasta ahora, he podido ejecutar el modelo de manera correcta

- **NOTA: Al ejecutar la función me sale un aviso de que estoy ejecutando una función costosa y me da los siguientes consejos para evitar estos cargos excesivos: Poner como argumento "reduce_tracing = True" o consultar documentación de TensorFlow [Doc1](https://www.tensorflow.org/guide/function#controlling_retracing) [Doc2](https://www.tensorflow.org/api_docs/python/tf/function)

In [1]:
#Librerias a utilizar en todo este proceso

#!pip install openl3

import openl3
from skimage.io import imread
import functions.general_functions as gf

#Los embeddings se pueden leer con numpy
import numpy as np

2024-07-29 20:59:51.228546: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#Pre cargamos el modelo, al hacerlo solo una vez no es necesario pre cargar el modelo cada vez que se va a utilizar
modelo = openl3.models.load_image_embedding_model(input_repr="mel256", content_type="music", embedding_size=512)

#Variable global, de donde obtenemos la ruta de las imágenes de entrenamiento y prueba
ruta_imagenes_train = gf.get_data_path('train_images')
ruta_imagenes_test = gf.get_data_path('test_images')

### 1.1 Aplicación directa del modelo

In [3]:
#Método para generar los embeddings de manera directa

#Nombre de la imagen a la cual aplicaremos el modelo
imagen_name = '993123.jpeg'

#Leemos la imagen
imagen1 = imread(ruta_imagenes_train + imagen_name)
#Generamos el embedding de la imagen de manera directa
emb = openl3.get_image_embedding(imagen1, content_type="env", input_repr="linear", embedding_size=512)

emb

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step


array([[ 1.175706  ,  1.5434446 ,  1.8470824 ,  1.6111743 ,  3.9059458 ,
         1.6161033 ,  1.1251589 ,  1.8484113 ,  1.2820483 ,  1.2422009 ,
         1.2345903 ,  0.83172244,  1.2606227 ,  1.6085217 ,  2.310515  ,
         1.9807837 ,  2.63912   ,  1.8400848 ,  1.7005421 ,  1.7075837 ,
         1.0354801 ,  2.1518202 ,  0.6044424 ,  1.4306686 ,  0.98116654,
         0.777962  ,  3.3654976 ,  4.162442  ,  1.9882624 ,  0.7811913 ,
         2.5927725 ,  1.8348336 ,  1.7911009 ,  1.8612864 ,  2.2643867 ,
         2.5106506 ,  1.129749  ,  0.7803635 ,  1.5808517 ,  2.0452437 ,
         0.7477303 ,  2.566805  ,  1.2202104 ,  2.673956  ,  1.3030437 ,
         0.9613706 ,  1.4589942 ,  1.1933473 ,  1.6517575 ,  1.4095986 ,
         1.3867158 ,  1.8570193 ,  3.5165267 ,  1.0719959 ,  0.7293594 ,
         2.3112679 ,  0.84064364,  2.1612198 ,  3.0060468 ,  1.9224309 ,
         1.1812272 ,  1.891209  ,  2.472405  ,  1.2888657 ,  1.6927787 ,
         2.1506999 ,  1.3459386 ,  2.0038981 ,  1.5

### 1.2 Guardar Embedding para uso futuro

In [4]:
#Aplicación del modelo a más de una imagen

#Rutas finales de las imágenes a procesar

imagen2 = ruta_imagenes_train + '998892.jpeg'
imagen3 = ruta_imagenes_train + '994535.jpeg'
#Lista para guardar todas las imágenes
imagen_array = [imagen2, imagen3]
#Método para guardar los embeddings de cada imagen en la misma carpeta de donde vienen las imágenes
openl3.process_image_file(imagen_array, batch_size = 32)

openl3: Processing /Users/pedrovela/Docs/Datasets - ML/planttraits2024/train_images/998892.jpeg (1/2)
openl3: /Users/pedrovela/Docs/Datasets - ML/planttraits2024/train_images/998892.npz exists, skipping.
openl3: Processing /Users/pedrovela/Docs/Datasets - ML/planttraits2024/train_images/994535.jpeg (2/2)
openl3: /Users/pedrovela/Docs/Datasets - ML/planttraits2024/train_images/994535.npz exists, skipping.


In [5]:
#En este método se leerán los embeddings de las imágenes generadas en la sección anterior

#Cargamos la data (embedding) de la imagen especificada
data = np.load(ruta_imagenes_train + '998892.npz')
#Obtenemos solo el embedding
emb = data['embedding']

emb[0]

array([-0.17171964,  2.7841794 ,  0.58611256, ...,  0.55718577,
        1.9009621 ,  1.2161709 ], dtype=float32)

### 1.3 Uso del modelo pre cargado

In [ ]:
#Método en el cual generamos el embedding de la imagen ya con el modelo pre cargado

#Obtenemos el embedding final
emb = openl3.get_image_embedding(imagen1, model=modelo)
emb